# Deep Learning for Scene Classification: A Personal Project

## Introduction

Welcome to this Deep Learning project focused on scene classification. The primary objective is to build and evaluate Deep Learning models that can accurately classify images into one of the six predefined categories: Buildings, Forests, Glaciers, Mountains, Oceans, and Streets.

In this Jupyter Notebook, you'll find:

- An exploration of the dataset to better understand its structure and contents.
- Pre-processing steps, including image transforms and augmentations, tailored specifically for this dataset.
- Implementation of Deep Learning models from scratch to solve the classification problem.
- Evaluation of models using metrics like Accuracy, Precision, Recall, and F1 Score.
- Visualizations and logs that track model performance during training and validation phases.

This project is implemented locally, and Conda is used for package management, ensuring that all dependencies are correctly set up for anyone who wishes to reproduce this work.

Let's get started!


In [1]:
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used
print(f'Using device: {device}')


Using device: cuda
